In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torchvision import transforms, datasets
from torch.optim import lr_scheduler
import os

In [2]:
root_folder = os.path.dirname(os.getcwd())
path_to_train_data = path.join(root_folder,'data/train_data.npy');
path_to_labels = path.join(root_folder,'data/train_labels.npy');
path_to_mean_std = path.join(root_folder,'data/mean_std.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [3]:
num_classes = 2;
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 15000;
nb_of_epochs = 25;
logging_interval = 30
dataset_divider = 0.2
net = cnn.resnet18(num_classes = num_classes);
net.to(device)

ResNet(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size

In [4]:
data = np.load(path_to_train_data);
labels = np.load(path_to_labels);
torch_data = torch.tensor(data).float()
torch_labels = torch.tensor(labels).int()
train_data, test_data = cnn.GenerateTrainAndTestDataset(torch_data, torch_labels, 0.2)


In [5]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=8)




In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
for i in range(nb_of_epochs):  # loop over the dataset multiple times
  loss = cnn.Train(net, device, train_loader, optimizer, criterion, exp_lr_scheduler, i, logging_interval)
  model_name = 'models/resnet18_num_classes_' + str(num_classes) + "_epoch_" + str(i) + '.pt';
  path_to_model = path.join(root_folder, model_name)
  torch.save({'epoch': i,'model_state_dict': net.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss,}, path_to_model)
  cnn.Test(net, device, criterion, test_loader)

Train Epoch: 0 [0/3100826 (0%)]	Loss: 0.50448036193847656250
Train Epoch: 0 [450000/3100826 (14%)]	Loss: 0.30568408966064453125
Train Epoch: 0 [900000/3100826 (29%)]	Loss: 0.19846571981906890869
Train Epoch: 0 [1350000/3100826 (43%)]	Loss: 0.14989276230335235596
Train Epoch: 0 [1800000/3100826 (58%)]	Loss: 0.12462782859802246094
Train Epoch: 0 [2250000/3100826 (72%)]	Loss: 0.10832166671752929688
Train Epoch: 0 [2700000/3100826 (87%)]	Loss: 0.10019866377115249634

Test set: Average loss: 0.00000575543526792899, Accuracy: 753723/775206 (97.22873661968560554669%)

Train Epoch: 1 [0/3100826 (0%)]	Loss: 0.09844838827848434448
Train Epoch: 1 [450000/3100826 (14%)]	Loss: 0.09680142253637313843
Train Epoch: 1 [900000/3100826 (29%)]	Loss: 0.09492018818855285645
Train Epoch: 1 [1350000/3100826 (43%)]	Loss: 0.09200423955917358398
Train Epoch: 1 [1800000/3100826 (58%)]	Loss: 0.08385221660137176514
Train Epoch: 1 [2250000/3100826 (72%)]	Loss: 0.08008942008018493652
Train Epoch: 1 [2700000/3100826 (

In [ ]:
cnn.Test(net, device, criterion, test_loader)